In [8]:
# Necessary imports
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
import os
import neptune

In [9]:
# Variables needed for neptune.
proxies = {
 "http": "http://172.16.2.30:8080",
 "https": "http://172.16.2.30:8080",
}

project_name='Hatespeech-CNERG/MH-Bert'
api_token = 'Api token hidden for security purposes'


In [10]:
# Initialize neptune for logging metrics
neptune.init(project_name,api_token=api_token,proxies=proxies)
neptune.set_project(project_name)

Project(Hatespeech-CNERG/MH-Bert)

In [11]:
# LASER embeddings for each data point are 1024 dimensional, with the ith value stored as ith column in the 
# csv file
cols = [str(i) for i in range(1024)]

In [12]:
# The languages on which we are running the baseline experiment.
langs = ['Arabic','English','German','Indonesian','Italian','Polish','Portugese','Spanish','French']

In [13]:
#Function to run baseline on sampled dataset
# Parameters: 
#   lang - language to train and test on
#   sample - sample_ratio
#   c - a hyperparameter for the logistic regression model. 
#       Refer to sklearn logistic regression docs (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
#       for explanation of this value
#   samp_type - the way in which data points are sampled from the training data - stratified in our case
#   seed_val - for setting the random seed value for reproducibility.

def train_and_test_baseline(lang,sample,c=1,samp_type='stratified',seed_val=2018):
    # Define the parameters of the experiment and create a neptune experiment with these parameters
    params = {}
    params['how_train'] = 'baseline'
    params['random_seed'] = seed_val
    params['sample_ratio'] = sample
    params['lang'] = lang
    params['samp_strategy'] = samp_type
    params['C'] = c
    name_one = lang+'_'+str(sample)+'_'+samp_type+'_'+str(seed_val)
    neptune.create_experiment(name_one,params=params,send_hardware_metrics=False,run_monitoring_thread=False)
    neptune.append_tag('LR one vs one baseline')
    neptune.append_tag(lang)
    neptune.append_tag('only_hate')
    neptune.append_tag(samp_type)
    # Load training dataset and split the embeddings and labels
    train=pd.read_csv(os.path.join('../Dataset/embedded/train',lang+'.csv'))
    train_x = train[cols]
    train_y = train['label']
    # If sample is needed, take the sample based on the strategy used.
    if samp_type=='equal':
        if sample==0.5:
            sample = int(len(train)/2)
        train_ = train.groupby('label',group_keys=False).apply(lambda x: x.sample(int(sample/2),random_state=seed_val).sample(frac=1,random_state=seed_val).reset_index(drop=True))
        train_x = train_[cols]
        train_y = train_['label']
    elif samp_type=='stratified':
        sss = StratifiedShuffleSplit(n_splits=1,test_size=sample,random_state=seed_val)
        a = list(sss.split(train_x,train_y))
        train_x = train_x.iloc[a[0][1],:]
        train_y = pd.DataFrame(train_y).iloc[a[0][1],:]
    # Read the test and validation data
    test = pd.read_csv(os.path.join('../Dataset/embedded/test',lang+'.csv'))
    test_x = test[cols]
    test_y = test['label']
    val = pd.read_csv(os.path.join('../Dataset/embedded/val',lang+'.csv'))
    val_x = val[cols]
    val_y = val['label']
    print(lang,c,sample,samp_type)
    # Initialize a logistic regression model. The class weights are taken as balanced to account for data imbalance
    model = LogisticRegression(C=c,solver='lbfgs',class_weight='balanced',random_state=seed_val)
    # Train the model on the dataset
    model.fit(train_x,train_y)
    print('Training complete!')
    # Get the predictions for the val and test sets
    pred = model.predict(test_x)
    fscore = f1_score(test_y,pred,average='macro')
    accuracy = accuracy_score(test_y,pred)
    pred_val = model.predict(val_x)
    val_fscore = f1_score(val_y,pred_val,average='macro')
    val_accuracy = accuracy_score(val_y,pred_val)
    print("Macro F1: ",'{0:.6f}'.format(f1_score(test_y,pred,average='macro')))
    print('')
    # Log the metrics accuracy and macro f1 score in neptune.
    neptune.log_metric('val_fscore',val_fscore)
    neptune.log_metric('val_acc',val_accuracy)
    neptune.log_metric('test_fscore',fscore)
    neptune.log_metric('test_accuracy',accuracy)
    neptune.stop()
    return fscore
        

In [14]:
# Run the above function for all languages, and 4 chosen c values, for 5 seed values each.
for lang in ['Arabic','German','Indonesian','Italian','Polish','Portugese','Spanish','French']:
    for c in [0.01,0.1,1,10]:
        for samp in [16,32,64,128,256]:
            for ty in ['stratified']:
                for seeds in [2018,2019,2020,2021,2022]:
                    seed_val = seeds
                    np.random.seed(seed_val)
                    train_and_test_baseline(lang,samp,c,ty,seed_val)

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-8012
French 10 16 stratified


/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training complete!
Macro F1:  0.504216

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-8014
French 10 16 stratified
Training complete!
Macro F1:  0.533393



/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-8015
French 10 16 stratified
Training complete!
Macro F1:  0.522182



/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-8016
French 10 16 stratified
Training complete!
Macro F1:  0.535655



/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-8018
French 10 16 stratified
Training complete!
Macro F1:  0.493741



/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
# Function to run baseline on 100% training dataset. 
def train_test_full(lang,c,seed_val):
    # Define the parameters of the experiment and create a neptune experiment with these parameters
    params = {}
    params['how_train'] = 'baseline'
    params['lang'] = lang
    params['sample_ratio'] = 100
    params['random_seed'] = seed_val
    params['C'] = c
    name_one= lang+'_'+'baseline'+'_100_'+str(seed_val)
    neptune.create_experiment(name_one,params=params,send_hardware_metrics=False,run_monitoring_thread=False)
    neptune.append_tag('LR one vs one baseline')
    neptune.append_tag(lang)
    neptune.append_tag('only_hate')
    # Load training dataset and split the embeddings and labels
    train = pd.read_csv(os.path.join('../Dataset/embedded/train',lang+'.csv'))
    train = train.sample(frac=1,random_state=seed_val).reset_index(drop=True)
    train_x = train[cols]
    train_y = train['label']
    # Initialize a logistic regression model. The class weights are taken as balanced to account for data imbalance
    model = LogisticRegression(C=c,solver='lbfgs',class_weight='balanced',random_state=seed_val)
    # Train the model on the dataset
    model.fit(train_x,train_y)
    # Read the test and validation datasets
    val = pd.read_csv(os.path.join('../Dataset/embedded/val',lang+'.csv'))
    val_x = val[cols]
    val_y = val['label']
    test = pd.read_csv(os.path.join('../Dataset/embedded/test',lang+'.csv'))
    test_x = test[cols]
    test_y = test['label']
    print(lang,c,seed_val)
    print('Training complete!')
    # Get the predictions for the val and test sets
    pred = model.predict(test_x)
    fscore = f1_score(test_y,pred,average='macro')
    accuracy = accuracy_score(test_y,pred)
    pred_val = model.predict(val_x)
    val_fscore = f1_score(val_y,pred_val,average='macro')
    val_accuracy = accuracy_score(val_y,pred_val)
    print("Macro F1: ",'{0:.6f}'.format(f1_score(test_y,pred,average='macro')))
    print('')
    # Log the metrics accuracy and macro f1 score in neptune.
    neptune.log_metric('val_fscore',val_fscore)
    neptune.log_metric('val_acc',val_accuracy)
    neptune.log_metric('test_fscore',fscore)
    neptune.log_metric('test_accuracy',accuracy)
    neptune.stop()
    return fscore

In [7]:
# Same setup as above, run the LR model for all languages and c values, for 5 different seed values
for lang in langs:
    for c in [0.01,0.1,1,10]:
        for seed_val in [2018,2019,2020,2021,2022]:
            np.random.seed(seed_val)
            train_test_full(lang,c,seed_val)

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6522
Arabic 0.01 2018
Training complete!
Macro F1:  0.698254

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6523
Arabic 0.01 2019
Training complete!
Macro F1:  0.698254

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6524
Arabic 0.01 2020
Training complete!
Macro F1:  0.698254

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6526
Arabic 0.01 2021
Training complete!
Macro F1:  0.698254

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6527
Arabic 0.01 2022
Training complete!
Macro F1:  0.698254

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6528
Arabic 0.1 2018
Training complete!
Macro F1:  0.753611

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6530
Arabic 0.1 2019
Training complete!
Macro F1:  0.753611

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6531
Arabic 0.1 2020
Training complete!
Macro F1:  0.753611

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6532
Arabic 0.1 2021
Training 

/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Arabic 10 2018
Training complete!
Macro F1:  0.792033

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6540


/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Arabic 10 2019
Training complete!
Macro F1:  0.792033

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6541


...

Spanish 10 2020
Training complete!
Macro F1:  0.699502

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6724


/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Spanish 10 2021
Training complete!
Macro F1:  0.700298

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6725


/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Spanish 10 2022
Training complete!
Macro F1:  0.699502

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6727
French 0.01 2018
Training complete!
Macro F1:  0.620770

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6728
French 0.01 2019
Training complete!
Macro F1:  0.620770

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6730
French 0.01 2020
Training complete!
Macro F1:  0.620770

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6731
French 0.01 2021
Training complete!
Macro F1:  0.620770

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6732
French 0.01 2022
Training complete!
Macro F1:  0.620770

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6733
French 0.1 2018
Training complete!
Macro F1:  0.617184

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6734
French 0.1 2019
Training complete!
Macro F1:  0.617184

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6735
French 0.1 2020
Training complete!
Macro F1:  0.617184

https://ui.neptune.ai/Hates